在unseen perturbation的case上跑linear model

env: scGPT

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

import pickle
import sys
import requests

from types import MethodType
import importlib
from scperturb import *

import anndata as ad

In [2]:
sys.path.append("/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/")

In [3]:
import v1
from v1.utils import *
from v1.dataloader import *

In [4]:
importlib.reload(v1)
importlib.reload(v1.utils)
importlib.reload(v1.dataloader)

<module 'v1.dataloader' from '/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/dataloader.py'>

# 所有的datasets

In [5]:
gene_pert_dict = \
{'AdamsonWeissman2016_GSM2406675_10X001': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 0},
 'AdamsonWeissman2016_GSM2406677_10X005': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 0},
 'AdamsonWeissman2016_GSM2406681_10X010': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 0},
#  'DatlingerBock2017': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 'DatlingerBock2021': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'DixitRegev2016': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'FrangiehIzar2021_RNA': {'perturbation_type': 'CRISPR-cas9', # 唯一一个multi celltype，先留着
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 
#  'LaraAstiasoHuntly2023_exvivo': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LaraAstiasoHuntly2023_invivo': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LaraAstiasoHuntly2023_leukemia': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LiangWang2023': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 1,
#   'multi_sgRNA': 1},
#  'McFarlandTsherniak2020': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 0},
 'NormanWeissman2019_filtered': {'perturbation_type': 'CRISPRa',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 0},
 'PapalexiSatija2021_eccite_RNA': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'PapalexiSatija2021_eccite_arrayed_RNA': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'ReplogleWeissman2022_K562_essential': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'ReplogleWeissman2022_K562_gwps': {'perturbation_type': 'CRISPRi', # pert太多了，先不测试
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 'ReplogleWeissman2022_rpe1': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'SantinhaPlatt2023': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 0},
 'ShifrutMarson2018': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'SunshineHein2023': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2019_day7neuron': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2019_iPSC': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2021_CRISPRa': {'perturbation_type': 'CRISPRa',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'TianKampmann2021_CRISPRi': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'XuCao2023': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1}}

In [6]:
prefix_list = list(gene_pert_dict.keys())

# load所有data并使用linear model得到结果

## 正式版

In [7]:
# - init para
data_dir = '/nfs/public/lichen/data/single_cell/perturb_data/scPerturb/raw/scPerturb_rna/statistic_20240520'
pert_cell_filter = 100 # this is used to filter perts, cell number less than this will be filtered
seed = 2024 # this is the random seed
split_type = 1 # 1 for unseen perts; 0 for unseen celltypes
split_ratio = [0.7, 0.2, 0.1] # train:test:val; val is used to choose data, test is for final validation
var_num = 5000 # selecting hvg number
num_de_genes = 20 # number of de genes
bs_train = 32 # batch size of trainloader
bs_test = 32 # batch size of testloader
lr = 1e-4

In [12]:
gene_embed_scgpt = pd.read_csv('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/utils_data/scGPT/gene_embed_scGPT.csv', sep = "\t")
gene_embed_scgpt

,RP5-973N23.5,RP11-182N22.10,CTB-53D8.3,RP11-348N17.2,RP11-205M20.8,RP11-326C3.17,RP11-439H13.3,RP11-413H22.3,GET1-SH3BGR,CH17-476P10.1,...,RP1-190J20.2,VN1R13P,RP11-155G14.6,RP1-152L7.5,RP11-525K10.1,RP11-390P2.2,SGSM3-AS1,RP11-812I20.2,AC022154.7,RBMXP1
0,-1.900666,1.556087,1.608870,1.130924,1.644218,0.835812,0.777010,-1.917366,-1.801478,1.117926,...,-0.093688,-0.339027,1.703432,-1.782005,0.212828,-0.133223,-1.700174,-0.117905,0.329495,0.856275
1,1.008205,1.778463,0.325701,-1.773786,-1.700228,-0.353182,-0.171027,0.407540,-0.322221,0.363011,...,0.914896,0.341640,-1.318827,0.009985,-1.272047,0.330320,-0.811589,-1.439252,0.524627,0.637495
2,1.700949,1.249432,1.779221,-0.930647,0.876516,-1.637807,1.379040,0.498992,-0.602994,1.509210,...,-0.844914,-1.485804,2.280034,1.073311,1.542383,1.274505,1.036156,1.922403,0.329258,1.544242
3,-1.543543,-0.428208,-1.611722,-1.764856,1.139952,-0.677079,-1.046558,-0.924131,-0.434652,-0.255988,...,1.401344,-1.801757,0.462960,1.128850,0.148551,0.962591,0.621725,-1.206026,-1.183687,2.225811
4,-0.146449,1.095613,1.605913,0.797817,0.231117,-1.523784,-0.800399,-0.923789,-0.259106,-1.258809,...,-0.997139,0.926254,-0.676847,1.403389,-0.806564,1.014099,-1.512837,0.483851,-1.159329,-0.494372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,-0.117272,-0.058358,-0.522179,-1.190935,-1.020210,-1.708623,-0.370135,-0.865606,0.675232,0.762173,...,1.294244,0.776270,0.999820,1.008411,0.496087,-1.114872,0.683504,-0.170933,0.259826,0.325960
508,1.193797,-1.304158,0.309257,1.560056,0.790933,-1.360069,-0.219487,-0.864897,0.193767,0.820187,...,-1.030318,0.581011,-0.567475,0.488943,-1.396035,1.285644,0.723238,2.002448,0.768177,0.681488
509,-0.771150,1.195878,1.909941,0.627506,-1.048198,1.567374,-1.280964,-0.002717,-0.862786,0.887900,...,1.494934,-0.878711,0.311551,0.089192,-0.694481,-0.082579,-0.408050,-0.235400,-0.972372,-1.271588
510,-0.558256,1.208881,1.899168,-1.391357,0.575956,-0.053123,1.969269,1.942591,0.417908,0.449813,...,1.585762,1.585318,0.401079,0.094355,-0.784628,0.684189,-0.598092,1.421850,0.376331,1.977252


In [13]:
linear_mode = 'scGPT' # training: G matrix is from the gene expression pca; 'scGPT': G is from scGPT; 'mean': just using the mean value b to predict

In [14]:

for prefix in prefix_list[:]:
    # prefix = 'NormanWeissman2019_filtered'
    print('='*20, prefix)

    fix_seed(2024)

    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/scPerturb'
    
    # - this is for load dataloader
    save_prefix = f'GEARS_v2-prefix_{prefix}-pert_cell_filter_{pert_cell_filter}-\
seed_{seed}-split_type_{split_type}-var_num_{var_num}-num_de_genes_{num_de_genes}-bs_train_{bs_train}-\
bs_test_{bs_test}'
    save_dir = os.path.join(tmp_dir, prefix, save_prefix)
    # - load pert_data
    pert_data = pickle.load(open(os.path.join(save_dir,'pert_data.pkl'), 'rb'))

    # - get go genes; special set for GEARS
    pert_data.get_gene2go()
    pert_data.set_pert_genes()
    pert_data.modify_gears(without_subgroup=True)
    
    
    #================================linear model part
    
    # - get the pca embedding of input genes; the mtx G
    adata_split = pert_data.adata_split

    
    
    if linear_mode == 'training' or linear_mode == 'mean':
        if adata_split.shape[0] > 5000:
            np.random.seed(2024)
            random_obs = np.random.choice(adata_split.obs_names, 5000, replace=False)
            gene_mtx = adata_split[random_obs].X
        else:
            gene_mtx = adata_split.X
            
        if not isinstance(gene_mtx, np.ndarray):
            gene_mtx = gene_mtx.toarray()
            
        from sklearn.decomposition import PCA

        # 设定需要降维的目标维度 n_components
        n_components = 512  # 你可以根据需要调整为其他值

        # 创建 PCA 对象并拟合数据
        pca = PCA(n_components=n_components, svd_solver='arpack', random_state=2024)
        gene_mtx_pca = pca.fit_transform(gene_mtx.T)
    elif linear_mode == 'scGPT':
        gene_list = pert_data.adata_split.var_names

        gene_embed = np.zeros((len(gene_list), 512))
        for i, gene in enumerate(gene_list):
            if gene in gene_embed_scgpt.columns:
                gene_embed[i, :] = gene_embed_scgpt.loc[:, gene].values
            else:
                np.random.seed(2024)
                gene_embed[i, :] = gene_embed_scgpt.iloc[:, np.random.randint(len(gene_list))].values
        gene_mtx_pca = gene_embed
    else:
        raise ValueError('linear_mode error')


    # - from the train pert get the gene expression; the mtx y_train
    cell_type = pert_data.filter_perturbation_list[0].split(' | ')[1]
    train_pert_groups = [i+' | '+cell_type for i in pert_data.train_perts]

    delta_mtx = []
    y_train = []
    for pert_group in train_pert_groups:
        adata_pert = pert_data.adata_split[pert_data.adata_split.obs['perturbation_group']==pert_group].copy()
        adata_ctrl = pert_data.adata_split[list(adata_pert.obs['control_barcode'])].copy()
        delta_mtx.append(np.array(np.mean(adata_pert.X-adata_ctrl.X, axis=0)).ravel())
        y_train.append(np.array(np.mean(adata_pert.X, axis=0)).ravel())
        
    delta_mtx = np.vstack(delta_mtx)
    Y_train = np.vstack(y_train).T

    # - get the pert embed; the mtx P
    def get_pert_mtx(gene_mtx_pca, perts):
        pert_mtx = []
        for pert in perts:
            if pert in adata_split.var_names:
                pert_mtx.append(gene_mtx_pca[list(adata_split.var_names).index(pert), :])
            else:
                np.random.seed(2024)
                pert_mtx.append(gene_mtx_pca[np.random.randint(0, gene_mtx_pca.shape[0]), :])
                
        pert_mtx = np.vstack(pert_mtx)
        return pert_mtx

    pert_mtx_train = get_pert_mtx(gene_mtx_pca, pert_data.train_perts)

    print(Y_train.shape, gene_mtx_pca.shape, pert_mtx_train.shape)

    # - infer the W
    b = np.mean(Y_train, axis=1).reshape(-1, 1)
    G = gene_mtx_pca
    P = pert_mtx_train

    lambda_reg = 0.1  # 岭回归的正则化参数

    # 计算 (Y_train - b)
    Y_centered = Y_train - b

    # 计算 (G^T G + λI)^-1
    G_T_G = G.T @ G
    lambda_I_G = lambda_reg * np.eye(G_T_G.shape[0])
    G_inv = np.linalg.inv(G_T_G + lambda_I_G)

    # 计算 (P^T P + λI)^-1
    P_T_P = P.T @ P
    lambda_I_P = lambda_reg * np.eye(P_T_P.shape[0])
    P_inv = np.linalg.inv(P_T_P + lambda_I_P)

    # 计算 W
    W = G_inv @ G.T @ Y_centered @ P @ P_inv

    # - use W to predict the result
    pert_mtx_test = get_pert_mtx(gene_mtx_pca, pert_data.test_perts)
    
    if linear_mode == 'mean':
        X_pred = b + G @ W @ pert_mtx_test.T - G @ W @ pert_mtx_test.T
    else:
        X_pred = b + G @ W @ pert_mtx_test.T

    pert_cat_list, pred_list, truth_list, pred_de_list, truth_de_list = [], [], [], [], []
    for i, pert in enumerate(pert_data.test_perts):
        # ====================================== generate out.pkl
        # - get test_res
        pert_group = pert + ' | ' + cell_type
        pert_gears = transform_name(pert_group)
        adata_pert = pert_data.adata_split[pert_data.adata_split.obs['perturbation_group']==pert_group]
        geneid2idx = dict(zip(pert_data.adata.var.index.values, range(len(pert_data.adata.var.index.values))))
        pert2pert_full_id = dict(pert_data.adata.obs[['condition', 'condition_name']].values)
        de_idx = [geneid2idx[j] for j in pert_data.adata.uns['rank_genes_groups_cov_all'][pert2pert_full_id[pert_gears]][:20]]

        # - add to list
        X_pred_tile = np.tile(X_pred[:, i], (adata_pert.shape[0], 1))
        pert_cat_list.append(np.array([pert_gears]*adata_pert.shape[0]))
        pred_list.append(X_pred_tile)
        truth_list.append(adata_pert.X.toarray())
        pred_de_list.append(X_pred_tile[:,de_idx])
        truth_de_list.append(adata_pert.X.toarray()[:, de_idx])

    test_res = {}
    test_res['pert_cat'] = np.hstack(pert_cat_list)
    test_res['pred'] = np.vstack(pred_list)
    test_res['truth'] = np.vstack(truth_list)
    test_res['pred_de'] = np.vstack(pred_de_list)
    test_res['truth_de'] = np.vstack(truth_de_list)
    # - get the out
    out = get_metric(pert_data.adata, test_res)
    # out_non_dropout = non_dropout_analysis(pert_data.adata, test_res)

    # - save the result
    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/unseen_perturbation/result/linear'
    save_prefix = f'linear_{linear_mode}'
    save_dir = os.path.join(tmp_dir, prefix, save_prefix)
    os.makedirs(save_dir, exist_ok=True)
    pickle.dump(out, open(os.path.join(save_dir,f'out.pkl'), 'wb'))
    # pickle.dump(out_non_dropout, open(os.path.join(save_dir,f'out_non_dropout.pkl'), 'wb'))
    
    # break

==================== AdamsonWeissman2016_GSM2406675_10X001


Found local copy...
Found local copy...


add adata finished
add condition finished
add set2conditions finished


/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


(5001, 4) (5001, 512) (4, 512)
get metrics... ...


...

Found local copy...
Found local copy...


add adata finished
add condition finished
add set2conditions finished


/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/ann

(5146, 118) (5146, 512) (118, 512)
get metrics... ...


  0%|          | 0/34 [00:00<?, ?it/s]/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: divide by zero encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: divide by zero encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: divide by zero encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: divide by zero encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: divide by zero encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byt

## debug

In [96]:
print(Y_train.shape, gene_mtx_pca.shape, pert_mtx_train.shape)

# - infer the W
b = np.mean(Y_train, axis=1).reshape(-1, 1)
G = gene_mtx_pca
P = pert_mtx_train

lambda_reg = 0.1  # 岭回归的正则化参数

# 计算 (Y_train - b)
Y_centered = Y_train - b

# 计算 (G^T G + λI)^-1
G_T_G = G.T @ G
lambda_I_G = lambda_reg * np.eye(G_T_G.shape[0])
G_inv = np.linalg.inv(G_T_G + lambda_I_G)

# 计算 (P^T P + λI)^-1
P_T_P = P.T @ P
lambda_I_P = lambda_reg * np.eye(P_T_P.shape[0])
P_inv = np.linalg.inv(P_T_P + lambda_I_P)

# 计算 W
W = G_inv @ G.T @ Y_centered @ P @ P_inv

# - use W to predict the result
pert_mtx_test = get_pert_mtx(gene_mtx_pca, pert_data.test_perts)
X_pred = b + G @ W @ pert_mtx_test.T


for i, pert in enumerate(pert_data.test_perts):
    # ====================================== generate out.pkl
    # - get test_res
    pert_group = pert + ' | ' + cell_type
    pert_gears = transform_name(pert_group)
    adata_pert = pert_data.adata_split[pert_data.adata_split.obs['perturbation_group']==pert_group]
    geneid2idx = dict(zip(pert_data.adata.var.index.values, range(len(pert_data.adata.var.index.values))))
    pert2pert_full_id = dict(pert_data.adata.obs[['condition', 'condition_name']].values)
    de_idx = [geneid2idx[j] for j in pert_data.adata.uns['rank_genes_groups_cov_all'][pert2pert_full_id[pert_gears]][:20]]

    # - add to list
    X_pred_tile = np.tile(X_pred[:, i], (adata_pert.shape[0], 1))
    pert_cat_list.append(np.array([pert_gears]*adata_pert.shape[0]))
    pred_list.append(X_pred_tile)
    truth_list.append(adata_pert.X.toarray())
    pred_de_list.append(X_pred_tile[:,de_idx])
    truth_de_list.append(adata_pert.X.toarray()[:, de_idx])

test_res = {}
test_res['pert_cat'] = np.hstack(pert_cat_list)
test_res['pred'] = np.vstack(pred_list)
test_res['truth'] = np.vstack(truth_list)
test_res['pred_de'] = np.vstack(pred_de_list)
test_res['truth_de'] = np.vstack(truth_de_list)
# - get the out
out = deeper_analysis_new(pert_data.adata, test_res)
out_non_dropout = non_dropout_analysis(pert_data.adata, test_res)

# - save the result
tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/scPerturb'
save_prefix = f'linear-prefix_{prefix}'
save_dir = os.path.join(tmp_dir, prefix, save_prefix)
os.makedirs(save_dir, exist_ok=True)
pickle.dump(out, open(os.path.join(save_dir,f'out.pkl'), 'wb'))
pickle.dump(out_non_dropout, open(os.path.join(save_dir,f'out_non_dropout.pkl'), 'wb'))

(5001, 4) (5001, 100) (4, 100)


/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:459: RuntimeWarning: divide by zero encountered in divide
  fold_change = pert_mean/ctrl
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:459: RuntimeWarning: invalid value encountered in divide
  fold_change = pert_mean/ctrl
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:703: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  val = fct(test_res['pred'][p_idx].mean(0)[de_idx], test_res['truth'][p_idx].mean(0)[de_idx])[0]
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:719: RuntimeWarning: divide by zero encountered in divide
  change_ratio

In [1]:
pert_mtx_train.shape

NameError: name 'pert_mtx_train' is not defined